In [3]:

import pyodbc

class DatabaseClient:
    def __init__(self):
        self.pyodbc_localhost()

    def pyodbc_localhost(self):
        driver = '{ODBC Driver 17 for SQL Server}'
        server = "localhost"
        database = "AIFMRM_ERS"
        cnxn_string = f"DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes;"
        try:
            cnxn = pyodbc.connect(cnxn_string)
        except Exception as e:
            print(f"Couldn't connect to db on localhost because {e}")
        else:
            self.cursor = cnxn.cursor()
            return cnxn_string


In [4]:
from flask import Flask
from flask_cors import CORS, cross_origin
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import pandas as pd

db_client = DatabaseClient()

cnxn_str = db_client.pyodbc_localhost()
cnxn_url = URL.create(
    "mssql+pyodbc", 
    query={"odbc_connect": cnxn_str}
)
engine = create_engine(cnxn_url)
class Database_df:
    def __init__(self, tables_df=pd.DataFrame(), table_name_list=[], select_template='', frames_dict={}):
        self.tables_df = tables_df
        self.table_name_list = table_name_list
        self.select_template = select_template
        self.frames_dict = frames_dict

    def create_dataframes(self, engine):
        # initialise connection via context manager           
        with engine.connect() as cnxn:
            self.tables_df = pd.read_sql('SELECT [name] AS [table_name] FROM sys.tables', cnxn)
            self.table_name_list = self.tables_df.table_name
            self.select_template = 'SELECT * FROM {table_name}'
            # Dictionary of table names and their respective SQL queries
            self.frames_dict = {}
            for tname in self.table_name_list:
                query = self.select_template.format(table_name = tname)
                self.frames_dict[tname] = pd.read_sql(query, cnxn)
            # Close db connection
            cnxn.close()
            # Return dictionary of dataframes
            return self.frames_dict

frames_dict = Database_df().create_dataframes(engine)


In [5]:
df_BA_Beta_Output = pd.DataFrame(frames_dict['tbl_BA_Beta_Output'])
df_Beta_Output = pd.DataFrame(frames_dict['tbl_Beta_Output'])
df_EOD_Equity_Data = pd.DataFrame(frames_dict['tbl_EOD_Equity_Data'])
df_EOD_Interest_Rate_Data = pd.DataFrame(frames_dict['tbl_EOD_Interest_Rate_Data'])
df_FTSEJSE_Index_Series = pd.DataFrame(frames_dict['tbl_FTSEJSE_Index_Series'])
df_Index_Constituents = pd.DataFrame(frames_dict['tbl_Index_Constituents'])
df_Industry_Classification_Benchmark = pd.DataFrame(frames_dict['tbl_Industry_Classification_Benchmark'])

In [7]:
import json
from flask import jsonify, request, Response

def getDates():
    dates_ls = df_Index_Constituents.Date.dt.strftime('%Y-%m-%d').unique().tolist()
    results_dict = {}
    for d in dates_ls:
        results_dict[d] = d
    
    resp = json.dumps(results_dict)
    
    # resp.status_code = 200 

    return resp
    
getDates()

'{"2017-09-18": "2017-09-18", "2017-12-18": "2017-12-18", "2018-03-19": "2018-03-19", "2018-06-18": "2018-06-18", "2018-09-25": "2018-09-25", "2018-12-24": "2018-12-24", "2019-03-18": "2019-03-18", "2019-06-24": "2019-06-24", "2019-09-23": "2019-09-23", "2019-12-23": "2019-12-23", "2020-03-23": "2020-03-23", "2020-06-22": "2020-06-22", "2020-09-21": "2020-09-21", "2020-12-21": "2020-12-21", "2021-03-23": "2021-03-23"}'

In [11]:
import calendar
from datetime import datetime

def monthEndDateValue(date):
    # Format = YYYY-MM-DD
    year = int(date[0:4])
    month = int(date[5:6])
    monthEndDate = calendar.monthrange(year, month)[1]
    return year, month, monthEndDate


def retEndDate(date):
    year, month, monthEndDate = monthEndDateValue(date)
    endDate = "-".join([str(year), str(month), str(monthEndDate)])
    return endDate

In [28]:
# print(df_Index_Constituents.shape)
def getIndexInstruments():
    # qry_args = request.args
    index_name = 'SAPY'
    date = '2017-9-18'
    endDate = retEndDate(str(date))
    index_name_search = '{0} New'.format(index_name)

    index_const_srt = df_Index_Constituents.set_index([index_name_search])
    res = index_const_srt.loc[index_name]
    # index_const_sub_df = index_const_srt.loc[(date, index_name):(endDate, index_name)]
    '''
    index_const_sub_df = index_const_sub_df[index_name_search == index_name]
    index_const_sub_df.reset_index(inplace=True)
    index_const_srt.set_index(['Instrument', 'Gross Market Capitalisation']).sort_index(level=["Gross Market Capitalisation"], ascending=[False])

    results_df = index_const_sub_df.loc[:, ['Instrument', 'Gross Market Capitalisation']]
    # results_df.sort_values(by=['Gross Market Capitalisation'], inplace=True)

    resp = results_df.to_json(orient='records') 
    '''
    return res

getIndexInstruments()

KeyError: 'SAPY'

In [59]:

# print(df_EOD_Equity_Data[df_EOD_Equity_Data["Instrument"] == 'ADR'])
# print(df_EOD_Equity_Data.shape)
# print(df_EOD_Equity_Data.columns)

def getSharePrice():

    instr = 'ADR'
    no_results = 10

    equity_data_srt = df_EOD_Equity_Data.set_index(['Instrument','Date']).sort_index(level=["Date"], ascending=[False])
    equity_index_sub_df = equity_data_srt.loc[[instr]]
    equity_index_sub_df.reset_index(inplace=True)
    
    results_df = equity_index_sub_df.loc[:, ['Date', 'Price']]
    
    return results_df.head(no_results)

getSharePrice()
    

,Date,Price
0,2021-03-31,620.0
1,2021-03-30,615.0
2,2021-03-29,625.0
3,2021-03-26,622.0
4,2021-03-25,625.0
5,2021-03-24,624.0
6,2021-03-23,624.0
7,2021-03-19,624.0
8,2021-03-18,629.0
9,2021-03-17,625.0
